In [ ]:

import sys
import os
import io

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import tensorflow as tf
import torchaudio.transforms as T
from torchaudio.datasets import SPEECHCOMMANDS

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm

import math
import tarfile
import multiprocessing

import numpy as np
import pickle
import requests
import pandas as pd
import time

import scipy
import librosa
#import boto3
#from botocore import UNSIGNED
#from botocore.config import Config

from IPython.display import Audio, display

In [ ]:
!pip3 install torchaudio

This approach was based on this paper. Convolutional Neural Networks for Raw Speech Recognition
Vishal Passricha and Rajesh Kumar Aggarwal http://dx.doi.org/10.5772/intechopen.80026



We use torchaudio to download and represent the dataset. Here we use SpeechCommands, which is a datasets of 35 commands spoken by different people. The dataset SPEECHCOMMANDS is a torch.utils.data.Dataset version of the dataset. In this dataset, all audio files are about 1 second long (and so about 16000 time frames long).

This part taken but changed from  https://pytorch.org/tutorials/intermediate/speech_command_recognition_with_torchaudio.html

A data point in the SPEECHCOMMANDS dataset is a tuple made of a waveform (the audio signal), the sample rate, the utterance (label), the ID of the speaker, the number of the utterance


In [ ]:
class DataSplitter(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("./", download=True)

        def load_list(filename):
            self.filepath = os.path.join(self._path, filename)
            with open(self.filepath) as fileobj:
                return [os.path.join(self._path, line.strip()) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]

In [ ]:
train_set = DataSplitter("training")
test_set = DataSplitter("testing")
val_set = DataSplitter("validation")

waveform, sample_rate, label, speaker_id, utterance_number = train_set[190]
waveform.shape

torch.Size([1, 16000])

In [ ]:
waveform, sample_rate, label, speaker_id, utterance_number

(tensor([[-1.8311e-04, -3.3569e-04, -9.1553e-05,  ..., -1.2512e-03,
          -3.0518e-04, -5.4932e-04]]), 16000, 'backward', '1e02ffc5', 3)

Here  several helper functions on how to deal the data for the Neaural Network. Not all will be used. A considerable EDA and FE on the data is to be seen on the eda_fe notebook. Here i decided to use the Transforms Classes from Pytorch
as i've understood they are similal to the Column Transformers from sklearn.

In [ ]:
def data_for_data_loader(data):
  '''A function to return the data for the DataLoader class to iterate when training the model'''

  for i in tqdm(range(64_747)):
    waveform, sample_rate, label, speaker_id, utterance_number = data[i]#train_set[i]
    #normalize the data
    waveform = (waveform - waveform.mean()) / waveform.std()
    mel_spectogram = create_melspectogram(waveform)
    label = 

  return mel_spectogram, label


def normalize_data(data):
  waveform_list = []

  for i in tqdm(range(64_747)):
    waveform, sample_rate, label, speaker_id, utterance_number = train_set[i]
    waveform = (waveform - waveform.mean()) / waveform.std()
    waveform_list.append((waveform, sample_rate, label))

  return waveform_list


def one_hot_encoded_labels(data):
  waveform_list = data_for_data_loader(data)

  labels_ = sorted(list(set(datapoint[2] for datapoint in waveform_list)))
  #if the labels are tensors!
  labels_ = torch.nn.functional.one_hot(labels_)
  return labels_

#functions taken from:
def label_to_index(word, data):
  waveform_list = data_for_data_loader(data)
  labels_ = sorted(list(set(datapoint[2] for datapoint in waveform_list)))
   # Return the position of the word in labels
  return torch.tensor(labels_.index(word))

def encode_labels(data):
    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number
  waveform_list = data_for_data_loader(data)
  labels_ = sorted(list(set(datapoint[2] for datapoint in waveform_list)))
  targets = []
  # Gather in lists, and encode labels as indices
  for label in labels_:
     targets += [label_to_index(label,data)]

  targets = torch.stack(targets)

  return targets

PyTorch works with DataLoader Classes -also with iterators and generators- that load the transformed data to the model. Here i have used in parts code and refactored it, changed it from different sources.
When uploaded to my public repository the sourses will be mentioned.

approach on label encodin form here https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html


In [ ]:
#PyTorch Transformers
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim

class TextTransformer:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('<SPACE>', ' ')


/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:433: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


In [ ]:
train_audio_transforms = nn.Sequential(
    #here changes-> sample rate
    torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_fft=1024, hop_length=512, n_mels=64)
    
#Even these transforms would improve acoording papers and documentation the performance of the model, i removed them after testing the model with these
#and getting errors.
    #torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    #torchaudio.transforms.TimeMasking(time_mask_param=100)
    #Here i apply also MFFC
    #torchaudio.transforms.MFFC(sample_rate= sample_rate,
                # n_mfcc= 40,
                 #dct_type = 2,
                 #norm = 'ortho',
                 #log_mels= True),
    #torchaudio.transforms.AmplitudeToDB(stype: str = 'power', top_db: Optional[float] = None)            
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram()
text_transform = TextTransform()

#These functions will be in the DataHandler Class
def transform_waveforms(data):
  '''A function that converts waveforms to Mel Spectograms'''

  for (waveform, sample_rate, label, speaker_id, utterance_number) in data:
    #function squeeze  removes dimensions
    spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
    waveform = (waveform - waveform.mean()) / waveform.std()
    spectrograms = []
    spectrograms.append(spec)
    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
#here to apply a log() for normalizing the data-> Its already made by the Transform class
  return spectrograms

def transform_labels(data):
  '''A function that converts the labels to tensors'''
  labels = []

  for (waveform, sample_rate, label, speaker_id, utterance_number) in data:
    label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
    labels.append(label)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

  return labels

def transform_data(data):
  '''Returns melspectograms and lables as 4 dimensional tensors'''
   spectrograms = []
   labels = []

   for (waveform, sample_rate, label, speaker_id, utterance_number) in data:
     waveform = (waveform - waveform.mean()) / waveform.std()
     spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
     spectrograms_.append(spec)
     label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
     labels.append(label)
     #as it seems the labels should be also tensors
     #This function here pads a list of variable length Tensors with padding_value pad_sequence stacks a list of Tensors along a new dimension, and pads them to equal length.
     spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
     labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

   return spectrograms, labels

def transform_waveform(waveform):
  '''Returns a melspectogram as a 4 dimensional tensor'''  
  train_audio_transforms = nn.Sequential(
  torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=1024,
        hop_length=512,
        n_mels=64)       
  )
  valid_audio_transforms = torchaudio.transforms.MelSpectrogram()
  text_transform = TextTransform()
  waveform = (waveform - waveform.mean()) / waveform.std()
  melspectogram = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
  return melspectogram


def waveform_to_melspectogram(data):
  '''Returns a melspectogram as a 2 dimensional tensor'''
  n_fft = 1024
  win_length = None
  hop_length = 512
  n_mels = 128
  for i in tqdm(range(64_747)):
    waveform, sample_rate, label, speaker_id, utterance_number = train_set[i]
    mel_spectrogram = T.MelSpectrogram(sample_rate=sample_rate, n_fft=n_fft, win_length=win_length, hop_length=hop_length, center=True,pad_mode="reflect",
                                       power=2.0, norm='slaney', onesided=True, n_mels=n_mels, mel_scale="htk",
)
    waveform = (waveform - waveform.mean()) / waveform.std()
    mel_spectogram = mel_spectrogram(waveform)

    return mel_spectogram

def create_melspectogram(waveform):
  n_fft = 1024
  win_length = None
  hop_length = 512
  n_mels = 128

  mel_spectrogram = T.MelSpectrogram(sample_rate=sample_rate, n_fft=n_fft, win_length=win_length, hop_length=hop_length, center=True,pad_mode="reflect",
                                     power=2.0, norm='slaney', onesided=True, n_mels=n_mels, mel_scale="htk")
  waveform = (waveform - waveform.mean()) / waveform.std()
  mel_spectogram = mel_spectrogram(waveform)

  return mel_spectogram


In [ ]:
waveform_to_melspectogram(train_set)

  0%|          | 0/64747 [00:00<?, ?it/s]


tensor([[[1.5496e+01, 1.9524e+02, 4.2929e+02,  ..., 8.7284e+00,
          1.7248e+02, 4.1024e+02],
         [3.8688e+00, 9.7259e+01, 1.5132e+02,  ..., 2.7743e+01,
          1.1723e+02, 8.7304e+01],
         [3.0970e+00, 2.0773e+01, 2.6702e+01,  ..., 5.0211e+01,
          4.3092e+01, 3.8962e+01],
         ...,
         [2.7885e-03, 1.1034e-03, 1.2200e-03,  ..., 7.5741e-04,
          1.4870e-03, 3.9908e-03],
         [7.2344e-04, 1.2109e-03, 9.4925e-04,  ..., 8.0535e-04,
          5.9425e-04, 1.2161e-03],
         [4.0984e-04, 3.2984e-04, 6.9453e-04,  ..., 6.2190e-04,
          8.3409e-04, 3.2910e-04]]])

In [ ]:
melspectogram = transform_waveform(waveform)
melspectogram

/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:433: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


tensor([[1.4177e-04, 7.1539e-05, 1.2141e-04,  ..., 2.8229e-04, 4.1490e-04,
         1.4119e-04],
        [3.9075e-05, 2.3011e-04, 5.2196e-04,  ..., 4.0095e-04, 3.6825e-04,
         1.3970e-04],
        [1.7551e-04, 7.7636e-05, 2.7296e-04,  ..., 2.5858e-04, 1.5001e-04,
         8.9354e-05],
        ...,
        [2.8391e-04, 7.9397e-04, 2.1794e-03,  ..., 8.4985e-04, 4.0586e-04,
         3.9696e-04],
        [5.8698e-04, 1.1963e-03, 6.0536e-04,  ..., 8.9311e-04, 1.0738e-03,
         2.9306e-04],
        [2.5178e-04, 9.2665e-04, 5.1814e-04,  ..., 5.5713e-04, 5.8877e-04,
         1.9919e-04]])

In [ ]:
mel_spectograms = transform_waveforms(train_set)
mel_spectograms.shape

torch.Size([1, 1, 64, 32])

I dont want a list of mel spectograms i want a spec pro time!

# Different ways on define the layers at Pytorch.

One way is this: the Sequeantial class which is a Container. It can be used also with the Transforms.The activation function is set here.

The other seperates the steps. The activation function is activated at the forward function called on a Functional "object".

In [ ]:
from torch import nn
from torchsummary import summary


class CNNNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        # 4 conv blocks / flatten / linear / softmax
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(128 * 5 * 4, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        logits = self.linear(x)
        predictions = self.softmax(logits)
        return predictions


if __name__ == "__main__":
    cnn = CNNNetwork()
    summary(cnn.cuda(), (1, 64, 44))

In [ ]:
from torchsummary import summary
import torch.nn.functional as F
import torch.nn as nn


class CNNNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=2)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=2)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()
        self.linear = nn.Linear(128 * 5 * 4, 10)
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = F.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.maxpool3(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.maxpool4(x)
        x = self.flatten(x)
        x = self.linear(x)
        #predictions = self.softmax(x)
        predictions = F.log_softmax(x)# here i can have a problems with the dimensions if so it can be given in the parameters ex dim=2
        return predictions


if __name__ == "__main__":
    cnn = CNNNetwork()
    summary(cnn.cpu(), (1, 64, 44))


Preparing your data for training with DataLoaders

The Dataset retrieves our dataset’s features and labels one sample at a time. While training a model, we typically want to pass samples in “minibatches”, reshuffle the data at every epoch to reduce model overfitting, and use Python’s multiprocessing to speed up data retrieval.

DataLoader is an iterable that abstracts this complexity for us in an easy API.

from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)


In [ ]:
#My appproach
from torch.utils.data import DataLoader

#Here i can use the RayTune hyperparameters.
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001


def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size)#train_data have a def __len__(self)!
    #here make a the data to be only melspectograms and the labels (in which form though????????)#As tensors or one hot encoded????
    return train_dataloader

#In Pytorch you write function for the train and test
#the batch size is here  in data_loader. My input data are the melspectograms and labels
def train_single_epoch(model, data_loader, loss_fn, optimiser, device):

  for batch_idx, data in enumerate(data_loader):
    #data here are the are the train_data = DataSplitter("training"), which the train_set that use in the data_for_data_loader fumnction to take the
    #mel_spectograms and labels. I don't use the function that returns a list because it would not iterate correct.
    #input , target = data_for_data_loader(data)#2 dim tensors...
    input , target = transform_data(data)
    input, target = input.to(device), target.to(device)

    # calculate loss
    #To the model i give the melspectogram and it expects 4-dimensional input for 4-dimensional weight [16, 1, 3, 3]
    prediction = model(input)
    loss = loss_fn(prediction, target)

        # backpropagate error and update weights
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()


def train(model, data_loader, loss_fn, optimiser, device, epochs, BATCH_SIZE = 128):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_single_epoch(model, data_loader, loss_fn, optimiser, device)
        print("---------------------------")
    print("Finished training")


if __name__ == "__main__":

#device in Pytorch is a specification on whether a GPU or CPU is used from the machine's user. If there is a GPU available then there is a CUDA selection.
#f.y.i. Macintosh with OS higher than HighSierra can't use the CUDA selection!
  if torch.cuda.is_available():
    device = "cpu"
  print(f"Using {device}")

    train_data = DataSplitter("training")
    test_data = DataSplitter("testing")
    val_data = DataSplitter("validation")
    
    train_dataloader = create_data_loader(train_set, BATCH_SIZE)

    # construct model and assign it to device
    cnn = CNNNetwork().to(device)
    print(cnn)

    # initialise loss funtion + optimiser
    loss_fn = nn.CTCLoss(blank=28).to(device)
    optimiser = torch.optim.Adam(cnn.parameters(),
                                 lr=LEARNING_RATE)

    # train model
    train(cnn, train_dataloader, loss_fn, optimiser, device, EPOCHS)

    #Try on one
    #train(cnn, melspectogram, loss_fn, optimiser, device, EPOCHS,BATCH_SIZE = 128)
    

    # save model
    torch.save(cnn.state_dict(), "feedforwardnet.pth")
    print("Trained feed forward net saved at feedforwardnet.pth")

Using cpu
CNNNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=2560, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)
Epoch 1


RuntimeError: ignored

In [ ]:
#To turn the int back to text so as to be able to read the prediction i used the Greedy Encoder. Much of this part are taken from DeepSpeech2 approach on Speech Recognition 
#models

#Test the model

A future approach on how to encode and decode the target
variable. The markov hypothesis based in Markov chains based on probabilities for each letter of the word. The CTC loss function i used above is a part of this  approach.

Here i took another approach, in which the model takes as an input a wav file that will be be wrangled the same way as the train, test and validation data and will map the word to a given dict with correct words.

In [ ]:
class_mapping = ['backward',
 'bed',
 'bird',
 'cat',
 'dog',
 'down',
 'eight',
 'five',
 'follow',
 'forward',
 'four',
 'go',
 'happy',
 'house',
 'learn',
 'left',
 'marvin',
 'nine',
 'no',
 'off',
 'on',
 'one',
 'right',
 'seven',
 'sheila',
 'six',
 'stop',
 'three',
 'tree',
 'two',
 'up',
 'visual',
 'wow',
 'yes',
 'zero']


def predict(model, input, target, class_mapping):
    model.eval()
    #
    with torch.no_grad():
      #This function here :Context-manager that disabled gradient calculation is disabling gradient calculation is useful for inference, when you are sure that you will not call Tensor.backward(). 
      #It will reduce memory consumption for computations that would otherwise have requires_grad=True.

        predictions = model(input)
        predicted_index = predictions[0].argmax(0)
        predicted = class_mapping[predicted_index]
        expected = class_mapping[target]
    return predicted, expected


if __name__ == "__main__":
    # load back the model
    cnn = CNNNetwork()
    state_dict = torch.load("cnnnet.pth")
    cnn.load_state_dict(state_dict)


    # get a sample from the SpeechCommands Dataset
    input, target = test_set[0][0], test_set[0][1] # [batch size, waveform, sample_rate etc]
    input.unsqueeze_(0)

    # make an inference
    predicted, expected = predict(cnn, input, target,
                                  class_mapping)
    print(f"Predicted: '{predicted}', expected: '{expected}'")
 

A